## Segmenting and Clustering Toronto

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

#This library can help us to get latitude and longitude from Postal Code.
import pgeocode

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
nomi = pgeocode.Nominatim('ca')
print(' las coordenadas son: lat:{}, lon:{}'.format(nomi.query_postal_code("M3A")[9],nomi.query_postal_code("M3A")[10]))

 las coordenadas son: lat:43.7545, lon:-79.33


In [43]:
URL = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(URL, match='Postal Code')
df_postcode_toronto = tables[0]
df_postcode_toronto = df_postcode_toronto[df_postcode_toronto['Borough'] != 'Not assigned']
df_postcode_toronto.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [50]:
# define the dataframe columns
column_names_toronto = ['Borough', 'Neighbourhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighbourhoods_toronto = pd.DataFrame(columns=column_names_toronto)


In [51]:
neighbourhoods_toronto

,Borough,Neighbourhood,Latitude,Longitude


In [52]:
nomi = pgeocode.Nominatim('ca')

for i in range(len(df_postcode_toronto)):
    postal_code = df_postcode_toronto["Postal Code"].iloc[i] 
    borough=df_postcode_toronto["Borough"].iloc[i] 
    neighbourhood=df_postcode_toronto["Neighbourhood"].iloc[i]
    if borough == 'Mississauga':
        address = 'Mississauga, Toronto'
        geolocator = Nominatim(user_agent="toronto_explorer")
        location = geolocator.geocode(address)
        latitude = location.latitude
        longitude = location.longitude
    else:
        latitude = nomi.query_postal_code(postal_code)[9]
        longitude = nomi.query_postal_code(postal_code)[10]
    neighbourhoods_toronto = neighbourhoods_toronto.append({'Borough': borough,'Neighbourhood': neighbourhood,'Latitude': latitude,'Longitude': longitude}, ignore_index=True)

In [58]:
neighbourhoods_toronto.head(20)

,Borough,Neighbourhood,Latitude,Longitude
0,North York,Parkwoods,43.7545,-79.3300
1,North York,Victoria Village,43.7276,-79.3148
2,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
3,North York,"Lawrence Manor, Lawrence Heights",43.7223,-79.4504
4,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
5,Etobicoke,"Islington Avenue, Humber Valley Village",43.6662,-79.5282
6,Scarborough,"Malvern, Rouge",43.8113,-79.1930
7,North York,Don Mills,43.7450,-79.3590
8,East York,"Parkview Hill, Woodbine Gardens",43.7063,-79.3094
9,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783


In [54]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [55]:
# add markers to map
for lat, lng, borough, neighbourhood in zip(neighbourhoods_toronto['Latitude'], neighbourhoods_toronto['Longitude'], neighbourhoods_toronto['Borough'], neighbourhoods_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighbourhoods in Downtown Toronto. 
So let's slice the original dataframe and create a new dataframe of the Downtown Toronto data.

In [148]:
Downtown_Toronto_data = neighbourhoods_toronto[neighbourhoods_toronto['Borough'] == 'Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto_data.head()

,Borough,Neighbourhood,Latitude,Longitude
0,Downtown Toronto,"Regent Park, Harbourfront",43.6555,-79.3626
1,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6641,-79.3889
2,Downtown Toronto,"Garden District, Ryerson",43.6572,-79.3783
3,Downtown Toronto,St. James Town,43.6513,-79.3756
4,Downtown Toronto,Berczy Park,43.6456,-79.3754


Let's get the geographical coordinates of Downtown Toronto

In [149]:
address = 'Downtown Toronto, TO'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto are 43.6541737, -79.38081164513409.


As we did with all of Toronto, let's visualizat Downtown Toronto the neighborhoods in it.

In [150]:
# create map of Manhattan using latitude and longitude values
map_downtown_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, label in zip(North_York_data['Latitude'], North_York_data['Longitude'], North_York_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_north_york)  
    
map_downtown_toronto